# Módulo: Normalización de Fechas
Este notebook contiene las funciones específicas para abordar y transformar las columnas de fechas según la estructura de cada archivo (Precios, Recaudo o Producción).

In [ ]:
import pandas as pd

In [ ]:
def map_spanish_months_to_num(series):
    """Mapea nombres de meses en español (texto) a números para facilitar conversión de fechas."""
    meses_map = {
        'enero': 1, 'febrero': 2, 'marzo': 3, 'abril': 4,
        'mayo': 5, 'junio': 6, 'julio': 7, 'agosto': 8,
        'septiembre': 9, 'octubre': 10, 'noviembre': 11, 'diciembre': 12
    }
    return series.astype(str).str.lower().str.strip().map(meses_map).fillna(series)

def normalizar_fecha(df, tipo_archivo):
    """
    Normaliza individualmente basado en sus columnas para sacar la respectiva fecha.
    """
    df_out = df.copy()
    
    if tipo_archivo == 'recaudo':
        col_fecha = [c for c in df_out.columns if 'mesrecaudo' in c.lower() or 'mes_recaudo' in c.lower()]
        if col_fecha:
            col = col_fecha[0]
            # Extraer mes y año partiendo el string por el guion "-"
            divisor = df_out[col].astype(str).str.split('-', n=1, expand=True)
            
            if divisor.shape[1] == 2:
                df_out['mes'] = map_spanish_months_to_num(divisor[0])
                df_out['año'] = divisor[1]
            else:
                df_out['mes'] = map_spanish_months_to_num(df_out[col])
                df_out['año'] = None
                
            df_temp_rec = pd.DataFrame({
                'year': pd.to_numeric(df_out['año'], errors='coerce'), 
                'month': pd.to_numeric(df_out['mes'], errors='coerce'), 
                'day': 1
            })
            df_out['fecha_normalizada'] = pd.to_datetime(df_temp_rec, errors='coerce').dt.date
            
    elif tipo_archivo in ['precios', 'precio']:
        # En el archivo de precios, tomamos la columna fecha3
        if 'fecha3' in df_out.columns:
            df_out['fecha_normalizada'] = pd.to_datetime(df_out['fecha3'], errors='coerce').dt.date
            
    elif tipo_archivo == 'produccion':
        # Utilizamos variables marcadas con 'año' en vez de 'ano'
        col_año = [c for c in df_out.columns if 'año' in c or 'ano' in c]
        col_mes = [c for c in df_out.columns if c == 'mes' or ('mes' in c and 'año' not in c and 'ano' not in c)]
        
        if col_año and col_mes:
            c_año = col_año[0]
            c_mes = col_mes[0]
            
            meses_num = map_spanish_months_to_num(df_out[c_mes])
            años = df_out[c_año]
            
            df_temp = pd.DataFrame({
                'year': pd.to_numeric(años, errors='coerce'), 
                'month': pd.to_numeric(meses_num, errors='coerce'), 
                'day': 1
            })
            df_out['fecha_normalizada'] = pd.to_datetime(df_temp, errors='coerce').dt.date
            
    return df_out